In [17]:
# DISCLAIMER: doesn't work
from tropy.learn import fit_hyperplane, fit_classifier
import pandas as pd

In [18]:
train_df = pd.read_csv('./data/smokers_train.csv').drop('id', axis=1)
test_df = pd.read_csv('./data/smokers_test.csv').drop('id', axis=1)
#train_df.insert(0, 'intercept', 0)
#test_df.insert(0, 'intercept', 0)

def z_scale(df):
    df.iloc[:, 1:] = (df.iloc[:, 1:] - df.iloc[:, 1:].mean()) / df.iloc[:, 1:].std()
    return df


In [19]:
def do_feature_eng(dataset):
    dataset = dataset.copy()
    dataset['BMI'] = dataset['weight(kg)'] / ((dataset['height(cm)'] / 100) ** 2)
    dataset['waist_height_ratio'] = dataset['waist(cm)'] / dataset['height(cm)']
    dataset['avg_eyesight'] = (dataset['eyesight(left)'] + dataset['eyesight(right)']) / 2
    dataset['avg_hearing'] = (dataset['hearing(left)'] + dataset['hearing(right)']) / 2
    dataset['cholesterol_ratio'] = dataset['HDL'] / dataset['LDL']
    return dataset

#train_df = do_feature_eng(train_df)
#test_df = do_feature_eng(test_df)

In [20]:
N = 5 # Keep the N most correlated features

correlation_matrix = train_df.corr()

top_correlations = correlation_matrix["smoking"].abs().nlargest(N + 1)
top_features = top_correlations.index

print("Top correlated features with", "smoking")
for feature, correlation in top_correlations.items():
    print(f"{feature}: {correlation}")

train_df = train_df[top_features]
test_df = test_df[top_features[1:]]
print(train_df['smoking'])

Top correlated features with smoking
smoking: 1.0
hemoglobin: 0.4506786564297312
height(cm): 0.44711105381189953
weight(kg): 0.35174789218573627
triglyceride: 0.33197510463307994
Gtp: 0.3055610317905156
0         1
1         0
2         1
3         0
4         1
         ..
159251    0
159252    0
159253    0
159254    1
159255    0
Name: smoking, Length: 159256, dtype: int64


In [21]:
train_plus = train_df[train_df['smoking'] == 1].iloc[:, 1:]
train_minus = train_df[train_df['smoking'] == 0].iloc[:, 1:]
#train_df = z_scale(train_df)
#test_df = z_scale(test_df)
Cplus_train = train_plus.to_numpy().T
Cminus_train = train_minus.to_numpy().T

C_test = test_df.to_numpy().T
print(Cplus_train.shape)
print(C_test.shape)

(5, 69653)
(5, 106171)


In [22]:
Cplus_trans, Cminus_trans, x, l = fit_hyperplane(Cplus_train, Cminus_train, N=100)
print("Apex:", x)
print("Margin:", abs(l))

Apex: [-306.14 -157.59 -229.54  116.81  576.46]
Margin: 0


In [23]:
predictor = fit_classifier(Cplus_trans, Cminus_trans, x, verbose=True)

[1 3 4] [0]


In [24]:
test_df_2 = pd.read_csv('./data/smokers_test.csv')
test_df_2['smoking'] = 0.5
for i, row in enumerate(C_test.T):
  test_df_2.at[i, 'smoking'] = predictor(row)
output_df = test_df_2[['id', 'smoking']]
output_df.to_csv('benchmark.csv', index=False)